# [유럽 호텔 리뷰 데이터](https://www.kaggle.com/datasets/jiashenliu/515k-hotel-reviews-data-in-europe)
> 이 데이터세트에는 유럽 전역의 1,493개 고급 호텔에 대한 515,000개의 고객 리뷰와 평점이 포함되어 있습니다. 또한, 추가 분석을 위해 호텔의 지리적 위치도 제공됩니다.

## 데이터 필드(컬럼) 설명

각 필드에 대한 설명은 다음과 같습니다.

- Hotel_Address
  - 호텔 주소.
- Review_Date
  - 리뷰어가 해당 리뷰를 게시한 날짜입니다.
- Average_Score
  - 지난 1년 동안의 최신 코멘트를 기반으로 계산된 호텔의 평균 점수입니다.
- Hotel_Name
  - 호텔 이름
- Reviewer_Nationality
  - 리뷰어의 국적
- Negative_Review
  - 리뷰어가 호텔에 남긴 부정적인 리뷰입니다.
  - 리뷰어가 부정적인 리뷰를 남기지 않은 경우, '부정적 리뷰 없음'으로 표시됩니다.
- Review_Total_Negative_Word_Counts
  - 부정적인 리뷰에 사용된 총 단어 수입니다.
- Positive_Review
  - 리뷰어가 호텔에 남긴 긍정적인 리뷰입니다.
  - 리뷰어가 부정적인 리뷰를 남기지 않은 경우, 'No Positive'로 표시됩니다.
- Review_Total_Positive_Word_Counts
  - 긍정적인 리뷰에 사용된 총 단어 수입니다.
- Reviewer_Score
  - 리뷰어가 호텔에 부여한 점수(리뷰어의 경험에 따른 점수)
- Total_Number_of_Reviews_Reviewer_Has_Given
  - 리뷰어가 과거에 작성한 리뷰 수입니다.
- Total_Number_of_Reviews
  - 호텔이 보유한 유효한 리뷰의 총 수입니다.
- Tags
  - 리뷰어가 호텔에 남긴 태그입니다.
- days_since_review
  - 검토 날짜와 스크래핑 날짜 사이의 기간.
- Additional_Number_of_Scoring
  - 리뷰를 남기지 않고 서비스에 대한 점수만 남긴 고객도 있습니다. 이 수치는 리뷰 없이 유효한 점수가 몇 개나 있는지를 나타냅니다.
- lat
  - 호텔의 위도
- lng
  - 호텔의 경도

# 데이터 로드

In [ ]:
import pandas as pd



## 데이터 확인

In [ ]:
# 전체 데이터의 수


In [ ]:
# 데이터의 컬럼 확인


In [ ]:
# 데이터 확인


# 조회 수 기반 추천

## 단순 인기도 기반 추천 단점
- 예를들면, 오늘 나온 뉴스는 사람들이 필요한 아이템이지만 아직 본 사람들이 없기 때문에 평점 또는 조회 수 값이 없기 때문에 추천에 포함되기 힘듭니다.
- 따라서 이런 경우에는 시간(등록일자)이 적용된 새로운 점수가 필요합니다.

`조회 수 기반 점수를 사용하게 되면 해당 문제를 해결할 수 있음`

## 1.신뢰도를 높이기 위해 조회 수가 충분히 많은 호텔들 조회하기
> 추천 신뢰도를 높이기 위해서 리뷰 수가 중위값보다 높은 호텔들 중에서 추천

In [ ]:
# 호텔별 리뷰 수 계산


# 리뷰 수의 중위값 계산


# 중위값보다 많은 리뷰를 가진 호텔 리스트 조회


# 중위값보다 많은 리뷰를 가진 호텔들만 적용


# 데이터 확인


## 2.조회 수 기반 점수 생성하기
- 목적(Hacker News Algorithm)
  - 오래됐지만 좋아요가 많은 게시물이 무조건 상단에 고정되지 않게 하여 정보의 신선도 유지
  - 반대로 시간은 조금 지났지만 빠르게 인기 있는 콘텐츠는 빠르게 상단 노출

### [2-1.Hacker News Algorithm 함수 만들기](https://medium.com/hacking-and-gonzo/how-hacker-news-ranking-algorithm-works-1d9b0cf2c08d)
- $pageviews$: 조회수
- $age$: 현재 시각 - 기사 업로드 날짜
- $gravity$: 중력상수(오래된 기사일 수록 score값을 작아지도록 조정하는 상수)
$$
Score = { pageviews -1 \over (age + 2)^{gravity} }
$$

### 2-2.pageviews(호텔별 조회 수) 구하기
- Hotel_Name
  - 호텔 이름

In [ ]:
# 호텔별 조회 수


### 2-3.age(얼마나 오래되었는지) 구하기

- days_since_review
  - 검토 날짜와 스크래핑 날짜 사이의 기간.

```python

# days_since_review는 얼마나 오래전에 리뷰를 작성했는지에 대한 데이터임   
hotel_enough_reviews['days_since_review']\
  # days_since_review의 값을 숫자로 변환
  .map(lambda x: int(x.replace('days','').replace('day','').replace(' ','')))

```

```python

# 호텔별
hotel_enough_reviews.groupby(['Hotel_Name'])\
  # 평균 리뷰 날짜
  .agg({'days_since_review':'mean'})\
  # 작성된 리뷰 날짜를 기준으로 정렬하기
  .sort_values(by='days_since_review', ascending=False).reset_index()

```

> days_since_review의 값은 위에서 작성한 코드에 의해서 평균 리뷰 날짜 데이터이다.   
> 따라서 컬럼명을 age로 변경할 수 있다.

### 2-4.pageviews & age 합치기

### 2-5.Hacker News Score 구하기
> hacker_news_score 함수를 이용하여 호텔별 Score 구하기

## 3.조회 수 기반 5개 호텔 추천하기

In [ ]:
no_ranking = 5 # 추천 수량


## 4.인기도와 조회 수를 적용한 호텔 추천하기
> 최신 조회도 중요하지만, 사용자들의 선호도 추천에서 중요한 지표다.

### 4-1.호텔별 선호도 추가하기

In [ ]:
# 호텔별 평균 리뷰 점수 추가하기


### 4-2.인기도와 조회 수를 적용하기 위해서 스케일(데이터 크기)를 조정하기

> 조회 수 점수(hacker_news_score)와 인기도 점수(Reviewer_Score)의 크기가 다르기 때문에   
> 단순히 덧샘을 하면 왜곡이 될 수 있음

> minmax scaling을 이용하여 각 점수들의 크기를 통일 시킴   
- minmax scaling이란
  - 최대값(max)를 1으로 변환 & 최소값(min)을 0으로 변환
  - 따라서 모든 값들을 0 ~ 1 사이로 크기를 변환하는 방법

$$
X_{scaled} = {X - X_{min} \over X_{max} - X_{min}}
$$

> 조회 수 점수(hacker_news_score) 스케일링하기

In [ ]:
# 최소값

# 최대값


# 스케일링 적용


# 결과 검증


> 인기도 점수(Reviewer_Score) 스케일링하기

In [ ]:
# 최소값

# 최대값


# 스케일링 적용


# 결과 검증


> hacker_news_score와 Reviewer_Score의 크기가 같아짐.

### 4-3.인기도와 조회 수를 적용하여 새로운 Score 구하기

> 각 항목별 중요도(weight)를 적용하여 Score 생성   
> 아래 코드는 인기도보다는 조회 수를 조금더 중요하게 적용한 코드임

In [ ]:
# hacker_wieght가 reviewer_weight보다 높게 적용하였음
# -> (해석) 평균 평점보다는 최근에 작성된 평점이 높은 호텔 추천


# 인기도와 조회 수를 적용한 새로운 score 생성하기


# 결과 확인하기


### 4-4.호텔 추천하기
- 단순 평균 평점으로 추천하게 된다면, Reviewer_Score의 값이 가장 높은 호텔이 첫번째로 추천되어야 한다.
- 하지만 hacker_new_score에 의해서 최근에 평가를 많이 받은 호텔들이 상위권에 추천된 것을 볼 수 있다.

In [ ]:
no_ranking = 5 # 추천 수량

